# Looking at the historical US market returns

## Imports

In [55]:
import pandas as pd
import matplotlib.pyplot as plt

## Data

### Shiller's historical S&P 500 data

In [56]:
df = pd.read_excel('../data/shiller_sp500.xls', sheet_name='Data', header=None, skiprows=8)

#### Manually adjust columns names

There's some weird formatting due to a preamble above the first several columns and some unused columns.  I chose slightly shorter names here for convenience and drop the columns with no actual data.

In [57]:
dropped_columns = ["No Data 1", "No Data 2"]

columns = [
    "Date",
    "S&P",
    "Dividend",
    "Earnings",
    "CPI",
    "Date Fraction",
    "Long Interest Rate",
    "Real Price",
    "Real Dividend",
    "Real Total Return Price",
    "Real Earnings",
    "Real TR Scaled Earnings",
    "CAPE",
    dropped_columns[0],
    "TR CAPE",
    dropped_columns[1],
    "Excess CAPE Yield",
    "Monthly Bond Returns",
    "Monthly Real Bond Returns",
    "10 Year Real Stock Return",
    "10 Year Real Bond Return",
    "10 Year Excess Return"
]

df.columns = columns

print("Was there any data in the dropped columns?")
filled_dropped_rows = df[~df[dropped_columns[0]].isna() | ~df[dropped_columns[1]].isna()]
print(len(filled_dropped_rows) > 0)

Was there any data in the dropped columns?
False
